In [1]:
import pandas as pd
import pickle
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from helper import *

/home/chakrapani/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### need to pass the user given input csv to embedding_generator and then use the embeddings here for X. y will be returned by the model as a number which can be converted to negative for 0 and positive for 1

In [2]:
# load data for inference
df = pd.read_csv("test_reviews_with_embeddings.csv")


In [13]:
pd.read_csv("sentiments_train_test_reviews/reviews_test_4000.csv").sample(20).to_csv("trial_20.csv")

#### preprocess data to feed to our models

In [3]:
df1 = pd.DataFrame()
df1 = df.dropna(subset=['embeddings'])#.sample(8000)
df1 = df1.copy()
df1['embeddings'] = df1.embeddings.apply(eval).apply(np.array)
new_df = pd.concat([pd.DataFrame(df1['embeddings'].to_list(), columns=[f"embed_{i}" for i in range(1536)])], axis=1)

new_df['label'] = [1 if value == 'positive' else 0 for value in df1['sentiment']]
print(new_df.shape)

(4000, 1537)


In [4]:
new_df.to_csv("final_test_set.csv",index=False)
new_df.head()

,embed_0,embed_1,embed_2,embed_3,embed_4,embed_5,embed_6,embed_7,embed_8,embed_9,...,embed_1527,embed_1528,embed_1529,embed_1530,embed_1531,embed_1532,embed_1533,embed_1534,embed_1535,label
0,-0.040045,-0.021374,-0.010932,-0.017637,-0.006115,0.034957,-0.001441,-0.015106,-0.016379,-0.030663,...,-0.004959,0.002700,-0.017637,-0.035858,-0.007785,-0.002289,-0.000832,-0.022328,0.003314,0
1,-0.009498,-0.029333,0.040726,-0.007752,-0.020466,0.000762,0.008822,-0.014151,-0.024191,-0.015466,...,-0.003825,0.024887,-0.026034,-0.036679,0.021794,0.010265,0.007752,-0.010040,-0.011915,0
2,-0.011595,-0.021313,-0.011504,-0.001785,-0.023543,0.006072,0.001181,-0.007324,-0.004415,-0.029425,...,-0.005830,0.016447,-0.013708,-0.033286,-0.005736,0.006616,0.002106,-0.017074,-0.008472,0
3,-0.004131,-0.011286,0.001132,-0.031751,-0.001232,0.009828,-0.005977,-0.052459,-0.026716,-0.039770,...,-0.020708,0.006993,-0.025204,-0.017671,0.054160,0.018778,-0.016524,-0.005876,-0.023813,1
4,-0.019083,-0.004487,-0.017896,-0.017712,-0.034184,-0.000612,-0.012944,-0.015852,-0.017145,-0.012067,...,0.011085,0.018041,-0.020218,-0.020415,0.015945,0.004316,-0.006179,-0.002118,-0.023712,1


In [5]:
X = new_df.drop("label", axis=1)
y_test = new_df['label']

### load models from pickle files 

In [6]:
# Load each model for inference
# XGBoost
with open("xgb.pickle", "rb") as file:
    loaded_xgb = pickle.load(file)

# SVM
with open("svm.pickle", "rb") as file:
    loaded_svm = pickle.load(file)

# Logistic Regression
with open("lr.pickle", "rb") as file:
    loaded_lr = pickle.load(file)

# Ensemble
with open("ensemble.pickle", "rb") as file:
    loaded_ensemble = pickle.load(file)


#### perform predictions

In [7]:

# XGBoost
xgb_predictions = loaded_xgb.predict(X)

# SVM
svm_predictions = loaded_svm.predict(X)

# Logistic Regression
lr_predictions = loaded_lr.predict(X)

# Ensemble
ensemble_predictions = loaded_ensemble.predict(X)


#### Accuracy scores

In [8]:

#Calculate accuracy for each
xgb_accuracy = accuracy_score(y_test, xgb_predictions)
svm_accuracy = accuracy_score(y_test, svm_predictions)
lr_accuracy = accuracy_score(y_test, lr_predictions)
ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)

print("XGBoost Accuracy:", xgb_accuracy)
print("SVM Accuracy:", svm_accuracy)
print("Logistic Regression Accuracy:", lr_accuracy)
print("Ensemble Accuracy:", ensemble_accuracy)

XGBoost Accuracy: 0.8835
SVM Accuracy: 0.92025
Logistic Regression Accuracy: 0.901
Ensemble Accuracy: 0.926


#### Classification report

In [9]:
print(classification_report(y_test , ensemble_predictions))

              precision    recall  f1-score   support

           0       0.89      0.96      0.93      1981
           1       0.96      0.89      0.92      2019

    accuracy                           0.93      4000
   macro avg       0.93      0.93      0.93      4000
weighted avg       0.93      0.93      0.93      4000



In [13]:
len(ensemble_predictions)

1000

In [15]:
results_df = pd.DataFrame()
results_df['review'] = df1['review']
results_df['sentiment'] = ["positive" if value == 1 else "negative" for value in ensemble_predictions]
results_df.to_csv("results.csv",index=False)

In [ ]:
training_results = convertDF2JSON(pd.read_csv("results.csv"))
training_results

In [11]:
prediction = 1
sentiment = 'positive' if prediction == 1 else 'negative'
sentiment

'positive'